In [1]:
import pandas as pd
df = pd.read_csv('/kaggle/input/train-dataset/updated_file_50001.csv')
df = df.drop(columns = ['group_id','image_link'])
df.head(5)

/tmp/ipykernel_36/2253302894.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/kaggle/input/train-dataset/updated_file_50001.csv')


,entity_name,entity_value,extracted_text
0,item_weight,2.0 gram,TRODUCT INFORMAHION eiaht.
1,item_weight,1.7 gram,2cm 1.7cm 1.7g
2,item_weight,10 kilogram,FORYPETS HYPOALLERGENIC JAMES HEALTHY NATURALL...
3,item_weight,10 kilogram,SPETS FOR. HYPOALLERGENIC JAMES HEALTHY NATURA...
4,maximum_weight_recommendation,1260 gram,11CM Shaped Sponge Cushion Comfortable Rebound...


In [2]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
import pandas as pd
from datasets import Dataset

train_data = [{'input_text': str(row['extracted_text']), 'target_text': str(row['entity_value'])} for _, row in df.iterrows()]

tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = T5ForConditionalGeneration.from_pretrained('t5-small')

def preprocess_function(examples):
    inputs = tokenizer(examples['input_text'], max_length=512, truncation=True, padding='max_length', return_tensors='pt')
    targets = tokenizer(examples['target_text'], max_length=128, truncation=True, padding='max_length', return_tensors='pt')
    inputs['labels'] = targets['input_ids']
    return inputs


train_df = pd.DataFrame(train_data)
train_df['input_text'] = train_df['input_text'].astype(str)
train_df['target_text'] = train_df['target_text'].astype(str)

dataset = Dataset.from_pandas(train_df)

tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=['input_text', 'target_text'])

training_args = TrainingArguments(
    output_dir='./results',            
    per_device_train_batch_size=16, 
    num_train_epochs=5,                 
    weight_decay=0.01,            
    logging_dir='./logs',              
    logging_steps=10,                   
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset)

trainer.train()

model.save_pretrained("text_extraction_model")
tokenizer.save_pretrained("text_extraction_model")

print("Model and tokenizer saved successfully.")


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

...

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
10,9.963400
20,3.784000
30,1.987400
40,1.400700
50,0.956600
60,0.684000
70,0.503700
80,0.397600
90,0.342900
100,0.277400


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0

Model and tokenizer saved successfully.


In [3]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import pandas as pd
import torch

tokenizer = T5Tokenizer.from_pretrained("/kaggle/working/text_extraction_model")
model = T5ForConditionalGeneration.from_pretrained("/kaggle/working/text_extraction_model")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

def generate_prediction(text, model, tokenizer):
    if not isinstance(text, str):
        raise ValueError("Input text must be a string")

    inputs = tokenizer(text, return_tensors='pt', max_length=512, truncation=True).to(device)

    with torch.no_grad():
        output = model.generate(**inputs)

    decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)
    return decoded_output

example_text = 'Peiwre blanc Campet White Kampot pepper Cambodia 70 grammes-2.470Z GRAND CRU'

prediction = generate_prediction(example_text, model, tokenizer)
print(f'Predicted value: {prediction}')

# df = pd.read_csv('/kaggle/input/train-dataset/updated_file_50001.csv')

# if 'extracted_text' in df.columns:
#     random_row = df.sample(1).iloc[0]
#     random_text = random_row['extracted_text']
    
#     random_text = str(random_text) if pd.notna(random_text) else ""
    
#     prediction_for_random = generate_prediction(random_text, model, tokenizer)
#     print(f'Prediction for random text: {prediction_for_random}')
# else:
#     print("The 'extracted_text' column is missing from the DataFrame.")


Predicted value: 7.0 gram


/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [2]:
# import pandas as pd
# import re
# entity_unit_map = {
#     'width': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
#     'depth': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
#     'height': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
#     'item_weight': {'gram', 'kilogram', 'microgram', 'milligram', 'ounce', 'pound', 'ton'},
#     'maximum_weight_recommendation': {'gram', 'kilogram', 'microgram', 'milligram', 'ounce', 'pound', 'ton'},
#     'voltage': {'kilovolt', 'millivolt', 'volt'},
#     'wattage': {'kilowatt', 'watt'},
#     'item_volume': {'centilitre', 'cubic foot', 'cubic inch', 'cup', 'decilitre', 'fluid ounce', 'gallon', 'imperial gallon', 'litre', 'microlitre', 'millilitre', 'pint', 'quart'}
# }

# df.dropna(subset=['extracted_text'], inplace=True)

# def create_pattern(entity_units):
#     units_pattern = '|'.join(map(re.escape, entity_units)) 
#     return r'(\d+\.?\d*)\s?(' + units_pattern + ')'

# def extract_value(text, entity_name):
#     if pd.isna(text):
#         return None
#     if entity_name in entity_unit_map:
#         pattern = create_pattern(entity_unit_map[entity_name])
#         matches = re.findall(pattern, text, re.IGNORECASE)
#         if matches:
#             results = [(float(match[0]), match[1].lower()) for match in matches]
#             if results:
#                 return results[0][0], results[0][1] 
#     return None

# df[['extracted_value', 'extracted_unit']] = df.apply(
#     lambda row: pd.Series(extract_value(row['extracted_text'], row['entity_name'])), axis=1
# )

# # Format the combined value and unit
# def format_value(value):
#     if pd.isna(value):
#         return None
#     elif value.is_integer(): 
#         return f"{int(value)}"  
#     else:
#         return f"{value}" 

# df['formatted_extracted_value'] = df.apply(
#     lambda row: f"{format_value(row['extracted_value'])} {row['extracted_unit']}" if pd.notna(row['extracted_value']) else None, axis=1
# )

# df.dropna(subset=['formatted_extracted_value'], inplace=True)
# df.to_csv('processed_data.csv', index=False)


In [3]:
# df.head(30)

In [4]:
# new_df = df[(df['entity_value'] == '2.47 ounce') & (df['entity_name'] == 'item_weight')]
# new_df

In [5]:
# new_df['extracted_text'].iloc[:-1].values
# new_df['extracted_text'].iloc[0:].values

In [6]:
# train_data = [{'input_text': row['extracted_text'], 'target_text': row['entity_value']} for _, row in df.iterrows()]